In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sponge_bob_magic.session_handler import State

spark = State().session
spark

In [3]:
from sponge_bob_magic.datasets import MovieLens

data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [4]:
from sponge_bob_magic.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance",
        "timestamp": "timestamp"
    }
)

In [5]:
from sponge_bob_magic.splitters import UserSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=1000,
    seed=1234,
    shuffle=True
)

In [6]:
from sponge_bob_magic.metrics import HitRate, NDCG, Coverage, Unexpectedness
from sponge_bob_magic.models import RandomPop
from sponge_bob_magic.scenarios import MainScenario

scenario = MainScenario(
    splitter=splitter,
    recommender=RandomPop(),
    criterion=HitRate,
    metrics={HitRate: 10, Coverage: 10},
    fallback_rec=None
)

/Users/shminke-ba/projects/sponge-bob-magic/venv/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [7]:
%%time

scenario.research(
    params_grid={"alpha": {"type": "uniform", "args": [-1, 10]}},
    log=log,
    k=10,
    n_trials=10
)

06-Apr-20 17:56:01, sponge_bob_magic, DEBUG: Деление лога на обучающую и тестовую выборку
06-Apr-20 17:56:20, sponge_bob_magic, DEBUG: Длина трейна и теста: 999209 1000
06-Apr-20 17:56:29, sponge_bob_magic, DEBUG: Количество пользователей в трейне и тесте: 6040, 1000
06-Apr-20 17:56:45, sponge_bob_magic, DEBUG: Количество объектов в трейне и тесте: 3706, 663
06-Apr-20 17:56:45, sponge_bob_magic, DEBUG: Инициализация метрик
06-Apr-20 17:56:51, sponge_bob_magic, DEBUG: Обучение и предсказание дополнительной модели
06-Apr-20 17:56:51, sponge_bob_magic, DEBUG: Пре-фит модели
06-Apr-20 17:56:56, sponge_bob_magic, DEBUG: -------------
06-Apr-20 17:56:56, sponge_bob_magic, DEBUG: Оптимизация параметров
06-Apr-20 17:56:56, sponge_bob_magic, DEBUG: Максимальное количество попыток: 100 (чтобы поменять его, задайте параметр 'optuna_max_n_trials')
06-Apr-20 17:56:56, sponge_bob_magic, DEBUG: -- Параметры: {'alpha': 7.502244802166226}
06-Apr-20 17:56:56, sponge_bob_magic, DEBUG: -- Сохраняем optuna

06-Apr-20 18:04:21, sponge_bob_magic, DEBUG: -- Дополняем рекомендации fallback рекомендациями
06-Apr-20 18:04:39, sponge_bob_magic, DEBUG: -- Длина рекомендаций: 10000
06-Apr-20 18:04:39, sponge_bob_magic, DEBUG: -- Подсчет метрики в оптимизации
06-Apr-20 18:04:58, sponge_bob_magic, DEBUG: HitRate=0.01
[I 2020-04-06 18:04:58,233] Finished trial#9 resulted in value: 0.006. Current best value is 0.013 with parameters: {'alpha': 6.384281804067535}.
06-Apr-20 18:04:58, sponge_bob_magic, DEBUG: Лучшее значение метрики: 0.01
06-Apr-20 18:04:58, sponge_bob_magic, DEBUG: Лучшие параметры: {'alpha': 6.384281804067535}


CPU times: user 7.43 s, sys: 1.84 s, total: 9.27 s
Wall time: 8min 56s


{'alpha': 6.384281804067535}

In [8]:
scenario.experiment.pandas_df.sort_values("HitRate@10", ascending=False)

,Coverage@10,HitRate@10
RandomPop({'alpha': 6.384281804067535}),0.652455,0.013
RandomPop({'alpha': 2.434136489801158}),0.614139,0.011
RandomPop({'alpha': 3.8350174032558444}),0.633297,0.011
RandomPop({'alpha': 2.3707447760240608}),0.617647,0.010
RandomPop({'alpha': 0.7346174555330363}),0.612250,0.009
RandomPop({'alpha': 9.506570116313906}),0.645170,0.008
RandomPop({'alpha': 7.502244802166226}),0.635456,0.007
RandomPop({'alpha': 2.8881473282018595}),0.627091,0.007
RandomPop({'alpha': 4.2387976768152935}),0.640583,0.006
RandomPop({'alpha': 9.210210630172496}),0.636266,0.006
